In [2]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..')))
from config import NEWS_API_KEY

import requests

In [4]:
url = ('https://newsapi.org/v2/everything?'
       'q=Apple&'
       'from=2024-11-01&'
       'sortBy=popularity&'
       f'apiKey={NEWS_API_KEY}')
response = requests.get(url)
articles = response.json().get('articles', [])

In [5]:
articles

[{'source': {'id': None, 'name': 'Yahoo Entertainment'},
  'author': 'Amy Skorheim',
  'title': 'Black Friday Apple deals 2024: The best Apple sales on iPads, AirPods, Apple Watches and MacBooks',
  'description': 'Apple devices have a few things in common: they’re well-designed, solidly built and easily connect with one another. But chances are, if it bears the Apple logo, it’s not cheap. That’s why promos like Black Friday are always worth checking out if you’re in th…',
  'url': 'https://consent.yahoo.com/v2/collectConsent?sessionId=1_cc-session_5b50f93c-f2e5-4f39-87c2-8667512a5599',
  'urlToImage': None,
  'publishedAt': '2024-11-19T16:51:43Z',
  'content': "If you click 'Accept all', we and our partners, including 237 who are part of the IAB Transparency &amp; Consent Framework, will also store and/or access information on a device (in other words, use … [+678 chars]"},
 {'source': {'id': None, 'name': 'Yahoo Entertainment'},
  'author': 'Amy Skorheim',
  'title': 'Black Friday Ap